Ref.
- https://github.com/google-research/google-research/blob/master/tft/script_train_fixed_params.py
- https://aihub.cloud.google.com/u/0/p/products%2F9f39ad8d-ad81-4fd9-8238-5186d36db2ec

In [1]:
!nvidia-smi

Thu Jun  4 05:02:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# install packages to download data and restart runtime
!pip install pyunpack
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=30437bd097ba19712bc05de6374b3bf1c0c11c5b3c7ec3aaf8d7257c57fff8c7
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


if the code is in goole drive, the following cell doesn't need to run.

In [0]:
# install to download only tft directory 
#!pip install git+git://github.com/HR/github-clone#egg=ghclone
#!ghclone https://github.com/google-research/google-research/tree/master/tft

In [0]:
from google.colab import drive

In [2]:
drive.mount('/Gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /Gdrive


In [3]:
cd '/Gdrive/My Drive/TFT'

/Gdrive/My Drive/TFT


In [0]:
#!ghclone https://github.com/google-research/google-research/tree/master/tft

In [5]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [6]:
cd tft

/Gdrive/My Drive/TFT/tft


In [0]:
# where $EXPT can be any of {volatility, electricity, traffic, favorita}, 
# and $OUTPUT_FOLDER denotes the root folder in which experiment outputs are saved.
EXPT='volatility'
OUTPUT_FOLDER='volatility_test'

In [8]:
# download $EXPT data
!python3 -m script_download_data $EXPT $OUTPUT_FOLDER

#### Running download script ###
Resetting data folder...
Getting volatility data...
Pulling data from https://realized.oxford-man.ox.ac.uk/images/oxfordmanrealizedvolatilityindices.zip to volatility_test/data/volatility/oxfordmanrealizedvolatilityindices.zip
done
Unzipping file: volatility_test/data/volatility/oxfordmanrealizedvolatilityindices.zip
Done.
Unzip complete. Adding extra inputs
/Gdrive/My Drive/TFT/tft/script_download_data.py:121: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  df['days_from_start'] = (dates - pd.datetime(2000, 1, 3)).days
Completed formatting, saving to volatility_test/data/volatility/formatted_omi_vol.csv
Done.
Download completed.


In [0]:
import argparse
import datetime as dte
import os

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

In [0]:
ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer

In [11]:
# args for electricity testing from gpu
name, output_folder, use_tensorflow_with_gpu = EXPT, OUTPUT_FOLDER, True

print("Using output folder {}".format(output_folder))

config = ExperimentConfig(name, output_folder)
formatter = config.make_data_formatter()

Using output folder volatility_test


In [0]:
expt_name=name
use_gpu=use_tensorflow_with_gpu
model_folder=os.path.join(config.model_folder, "fixed")
data_csv_path=config.data_csv_path
data_formatter=formatter
#use_testing_mode=False

In [0]:
#num_repeats = 1

if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
  raise ValueError(
      "Data formatters should inherit from" +
      "AbstractDataFormatter! Type={}".format(type(data_formatter)))

In [0]:
# Tensorflow setup
default_keras_session = tf.keras.backend.get_session()

In [15]:
if use_gpu:
  tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
  tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

print("*** Training from defined parameters for {} ***".format(expt_name))

Selecting GPU ID=0

*** Training from defined parameters for volatility ***


In [16]:
!pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=06abfcadaf5d4c8b9fa6faf15c01b348b4ca9aa2810be16cf69e7bb5545a150a
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [17]:
print("Loading & splitting data...")
raw_data = pd.read_csv(data_csv_path, index_col=0)
train, valid, test = data_formatter.split_data(raw_data)
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration(
)

Loading & splitting data...
Formatting train-valid-test splits.
Setting scalers with training data...
time: 1.29 s


In [18]:
# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()
params["model_folder"] = model_folder

time: 2.72 ms


In [19]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                    fixed_params, model_folder)

*** Loading hyperparm manager ***
time: 11.9 ms


In [20]:
# Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")
for k in params:
  print("{}: {}".format(k, params[k]))

*** Running calibration ***
Params Selected:
dropout_rate: 0.3
hidden_layer_size: 160
learning_rate: 0.01
minibatch_size: 64
max_gradient_norm: 0.01
num_heads: 1
stack_size: 1
model_folder: volatility_test/saved_models/volatility/fixed
time: 2.98 ms


In [21]:
params = opt_manager.get_next_parameters()

time: 1.71 ms


Next, we train a TFT model by subsampling the data in the training and validation sets.

In [0]:
tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a TFT model
    model = ModelClass(params, use_cudnn=True) # Run model on GPU using CuDNNLSTM cells

    # Sample data into minibatches for training
    if not model.training_data_cached():
        model.cache_batched_data(train, "train", num_samples=450000)
        model.cache_batched_data(valid, "valid", num_samples=50000)

    # Train and save model
    model.fit()
    model.save(model_folder)

Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 160
# learning_rate = 0.01
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = volatility_test/saved_models/volatility/fixed
# num_heads = 1
# stack_size = 1
# total_time_steps = 257
# num_encoder_steps = 252
# num_epochs = 100
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('Symbol', <DataTypes.CATEGORICAL: 1>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_vol', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('open_to_close', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('days_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_month', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('week_of_year', <DataTypes.CATEGORICAL: 1>, <InputTypes.KNOWN_INPUT: 2>), ('month

To evaluate model performance, we reload the serialised model and compute P50 and P90 losses.

In [0]:
tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a new model & load weights
    model = ModelClass(params, use_cudnn=True)
    model.load(model_folder)
    
    # Make forecasts
    output_map = model.predict(test, return_targets=True)

    targets = data_formatter.format_predictions(output_map["targets"])

    # Format predictions
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
          col for col in data.columns
          if col not in {"forecast_time", "identifier"}
        ]]

    # Compute Losses
    p50_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p50_forecast),
        0.5)
    p90_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p90_forecast),
        0.9)

In [0]:
print("Normalised quantile losses: P50={}, P90={}".format(p50_loss.mean(), p90_loss.mean()))

## Interpretability Use Cases
The relationships learnt by the TFT can also be studied using the trained model, through:

1. Analyzing the variable selection weights to identify significant features for the prediction problem.
2. Visualizing distributions of self-attention weights to determine the presence of any persistent temporal relationships.

In the remainder of this section, we demonstrate two interpretability use cases to showcase the above.

### Generate Weights for Interpretability
First, we generate all necessary variable selection and attention weights required for analysis.

In [0]:
# Store outputs in maps
counts = 0
interpretability_weights = {k: None for k in ['decoder_self_attn', 
                                              'static_flags', 'historical_flags', 'future_flags']}

tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a new model & load weights
    model = ModelClass(params, 
                                    use_cudnn=True)
    model.load(model_folder)
    for identifier, sliced in test.groupby('id'):
        
        print("Getting attention weights for {}".format(identifier))
        weights = model.get_attention(sliced)
        
        for k in interpretability_weights:
            w = weights[k]
            
            # Average attentin across heads if necessary
            if k == 'decoder_self_attn':
                w = w.mean(axis=0)
            
                # Store a single matrix for weights to reduce memory footprint
                batch_size, _, _ = w.shape                 
                counts += batch_size
            
            if interpretability_weights[k] is None:
                interpretability_weights[k] = w.sum(axis=0)
            else:
                interpretability_weights[k] += w.sum(axis=0)

interpretability_weight = {k: interpretability_weights[k]/counts for k in interpretability_weights}

print('Done.')

### Use Case 1: Analyzing Variable Importance 

Next, we analyze the distribution of variable selection weights on the input layer -- using this to quantify the relative importance of a given feature for the prediction problem in general. This is split into variable importance for static covariates, time-varying historical inputs variables and known future inputs as shown below.

In [0]:
import numpy as np
def get_range(static_gate, axis=None):
    """Returns the mean, 10th, 50th and 90th percentile of variable importance weights."""
    return {'Mean': static_gate.mean(axis=axis), 
               '10%': np.quantile(static_gate, 0.1, axis=axis),
               '50%': np.quantile(static_gate, 0.5, axis=axis),
               '90%': np.quantile(static_gate, 0.9, axis=axis)}

#### Static Variable Importance

In [0]:
def flatten(x):
    static_attn = x
    static_attn = static_attn.reshape([-1, static_attn.shape[-1]])
    return static_attn

static_attn = flatten(interpretability_weights['static_flags'])
m = get_range(static_attn, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['ID']) for k in m})

#### Temporal Variable Importance -- Past Inputs

In [0]:
x = flatten(interpretability_weights['historical_flags'])
m = get_range(x, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['Hour of Day', 'Day of Week', 'Time Index', 'Target']) for k in m})

#### Temporal Variable Importance -- Future Inputs

In [0]:
x = flatten(interpretability_weights['future_flags'])
m = get_range(x, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['Hour of Day', 'Day of Week', 'Time Index']) for k in m})

### Use Case 2: Visualizing Persistent Temporal Patterns
Lastly, we analyse the distribution of self-attention weights across various horizons to see if any persistent temporal patterns exist within the dataset. This allows us to identify any seasonal patterns or lagged relationships in the dataset, based on which past time steps are consistently important for predictions at a given horizon. 

We visualize this using the average attention pattern for various prediction horizons, as shown below:

#### Mean Attention Weights for Various Prediction Horizons

In [0]:
# Plotting libraries & Functions
import plotly.offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.graph_objs as go
import cufflinks as cf
from IPython.display import HTML

# Loads plotly charts
def iplot(fig, s='plot.html'):
    filename = os.path.join(output_folder, s)
    plotly.offline.plot(fig, filename=filename, auto_open=False)
    return HTML(filename)    

def plotly_chart(df, title=None, kind='scatter', x_label=None, y_label=None, secondary_y=None, fill=None,
                shape=None, subplots=False):
    
    fig = df.iplot(asFigure=True, title=title, kind=kind, xTitle=x_label, yTitle=y_label, secondary_y=secondary_y,
                  fill=fill, subplots=subplots, shape=shape)

    return iplot(fig)

In [0]:
self_attn = interpretability_weights['decoder_self_attn']

means = pd.DataFrame({"horizon={}".format(k): self_attn[model.num_encoder_steps+k-1, :] 
                      for k in [1, 5, 10, 15, 20]})
means.index -= model.num_encoder_steps

plotly_chart(means,
             x_label="Positiion Index (n)",
             y_label="Mean Attention Weight",
             title="Average Attention Pattern at Various Prediction Horizons")